In [1]:
import openfermion
#from openfermion import ran
import numpy as np
n_qubit = 6
n_electrons = 3

def is_particle_number_correct(arg: int, n_qubit: int, n_particle: int):
    l = bin(arg)[2:].zfill(n_qubit)
    return sum([int(b) for b in l]) == n_particle

# generate Haar random state
np.random.seed(1234)
vec_6q = np.random.normal(size = 2**n_qubit) + 1j * np.random.normal(size = 2**n_qubit)
vec_10q = np.random.normal(size = 2**10) + 1j * np.random.normal(size = 2**10)

# particle number restriction
vec_6q = [vec_6q[i] if is_particle_number_correct(i, 6, n_electrons) else 0 for i in range(2**6)]
vec_6q /= np.linalg.norm(vec_6q)

vec_10q = [vec_10q[i] if is_particle_number_correct(i, 10, n_electrons) else 0 for i in range(2**10)]
vec_10q /= np.linalg.norm(vec_10q)

## compute k-RDM

フェルミオン系の量子多体状態$\rho$に対して、その$k$-RDM (Reduced Density Matrix, 縮約密度行列)の行列要素を
\begin{align}
^k D^{p_1\cdots p_k}_{q_1 \cdots q_k} = \mathrm{Tr}[c_{p_1}^\dagger \cdots c_{p_k}^\dagger c_{q_1}\cdots c_{q_k} \rho]
\end{align}
により定義することにする。ここで、 $c_i^{(\dagger)}$は$i$番目のフェルミオンに関する消滅(生成)演算子を表す。

In [6]:
from openfermion import FermionOperator, get_sparse_operator

def expectation(operator, state):
    n_qubit = int(np.log2(state.shape[0]))
    if type(operator) == np.ndarray:
        return state.conj() @ operator @ state
    else:
        return state.conj() @ get_sparse_operator(operator, n_qubits = n_qubit).toarray() @ state
    


def get_sign_of_args(args):
    if len(args) - len(np.unique(args)) > 0:
        return 0
    order = [list(args).index(arg) for arg in np.sort(args)]
    return Permutation(order).signature()

def get_corresponding_index(args: tuple):
    k = len(args)//2
    assert len(args) == 2*k
    args_ = list(args)
    
    args1 = args_[:k]
    args2 = args_[k:]
    
    sign1 = get_sign_of_args(args1)
    sign2 = get_sign_of_args(args2)
    
    argmin1 = min(args1)
    argmin2 = min(args2)
        
    if_conjugate = argmin1 > argmin2
    if if_conjugate:
        args1_ref =  sorted(args2)
        args2_ref = sorted(args1)
    else:
        args1_ref = sorted(args1)
        args2_ref = sorted(args2)
    return tuple(args1_ref + args2_ref), if_conjugate, sign1 * sign2

    
from sympy.combinatorics import Permutation

def get_sign_of_args(args):
    if len(args) - len(np.unique(args)) > 0:
        return 0
    order = [list(args).index(arg) for arg in np.sort(args)]
    return Permutation(order).signature()

def get_corresponding_index(args: tuple):
    k = len(args)//2
    assert len(args) == 2*k
    args_ = list(args)
    
    args1 = args_[:k]
    args2 = args_[k:]
    
    sign1 = get_sign_of_args(args1)
    sign2 = get_sign_of_args(args2)
    
    argmin1 = min(args1)
    argmin2 = min(args2)
        
    if_conjugate = argmin1 > argmin2
    if if_conjugate:
        args1_ref =  sorted(args2)
        args2_ref = sorted(args1)
    else:
        args1_ref = sorted(args1)
        args2_ref = sorted(args2)
    return tuple(args1_ref + args2_ref), if_conjugate, sign1 * sign2


def compute_one_rdm(vector: np.ndarray):
    n_qubit = int(np.log2(vector.shape[0]))
    rdm1 = np.zeros((n_qubit, n_qubit), dtype = complex)
    for i in range(n_qubit):
        for j in range(i, n_qubit):
            cij = expectation(FermionOperator(f"{i}^ {j}"), vector)
            
            rdm1[i, j] = np.copy(cij)
            rdm1[j, i] = np.copy(cij).conj()
    return rdm1.copy()

def compute_two_rdm(vector: np.ndarray):
    n_qubit = int(np.log2(vector.shape[0]))
    rdm2 = np.zeros((n_qubit, n_qubit, n_qubit, n_qubit), dtype = complex)
    for i in range(n_qubit):
        for j in range(i, n_qubit):
            for k in range(j, n_qubit):
                for l in range(k, n_qubit):
                    
                    unique_args = list(set([(i, j, k, l), (i, k, j, l), (i, l, j, k)]))
                    for _args in unique_args:
                        exp = expectation(FermionOperator(f"{_args[0]}^ {_args[1]}^ {_args[2]} {_args[3]}"), vector)
                        rdm2[_args] = exp

    for i in range(n_qubit):
        for j in range(n_qubit):
            for k in range(n_qubit):
                for l in range(n_qubit):
                    
                    
                    args = (i, j, k, l)
                    args_ref, if_conjugate, sign = get_corresponding_index(args)
        
                    cijkl = np.copy(rdm2[args_ref]) * sign
                    if if_conjugate:
                        cijkl = cijkl.conj()

                    rdm2[i, j, k, l] = np.copy(cijkl)     
    return rdm2.copy()

In [7]:
%%time

rdm1 = compute_one_rdm(vec_6q)
rdm2 = compute_two_rdm(vec_6q)

CPU times: user 18.1 s, sys: 37.9 s, total: 56 s
Wall time: 9.68 s


In [8]:
%%time

# to be honest this is already slow
rdm1_10q = compute_one_rdm(vec_10q)
rdm2_10q = compute_two_rdm(vec_10q)

CPU times: user 4min 42s, sys: 9min 51s, total: 14min 34s
Wall time: 2min 35s


In [9]:
# openfermion内部では、2-RDMに定数倍をかけたものを活用している
tpdm = rdm2 / 2
tpdm_10q = rdm2_10q / 2

# openfermion内部では、1-RDMに定数倍をかけたものを活用している
opdm = (2 / (n_electrons - 1)) * np.einsum('ijjk', tpdm)
opdm_10q = (2 / (n_electrons - 1)) * np.einsum('ijjk', tpdm_10q)

In [10]:
# n-qubit系 (n-fermion mode系)では shape = (N, N)
opdm.shape

(6, 6)

In [11]:
# n-qubit系 (n-fermion mode系)では shape = (N, N, N, N)
tpdm.shape

(6, 6, 6, 6)

## wedge product of tensors



Wedge積は、反交換関係を保つようなKronecker deltaのようなものだと理解できる。
その定義は、以下のように与えられる：
\begin{align}
     \left( a \wedge b\right) ^{i_{1}, i_{2}, ..., i_{p},..., i_{p+q}}_{j_{1}, j_{2}, ...,j_{p}, ..., j_{p+q} } =
    \left(\frac{1}{N!}\right)^{2} 
\sum_{\pi, \sigma}
    \epsilon(\pi)
    \epsilon(\sigma)
    a_{\pi(j_{1}), \pi(j_{2}), ..., \pi(j_{p}) }^{ \sigma(i_{1}), \sigma(i_{2}), ..., \sigma(i_{p})}
    b_{\pi(j_{p+1}), \pi(j_{p+2}), ..., \pi(j_{p+q}) }^{ \sigma(i_{p+1}), \sigma(i_{p+2}), ..., \sigma(i_{p+q})}
    \end{align}
    ただし、$a$と$b$はそれぞれ $p$-RDM, $q$-RDMを表すテンソルで、フェルミオンの半交換関係に起因して行列要素同士が符号で結びついている。
    また、$\pi, \sigma$はそれぞれ $(p+q)$個の添字に関する巡回を表し、$\epsilon(\pi)$などは巡回操作の符号を表すものとする。
    
その素朴なeinsumによる実装は、googleの提供する Openfermionライブラリに提供されているが、例によって遅くて困っている。
`wedge`関数の[URL](https://quantumai.google/reference/python/openfermion/linalg/wedge) 

In [12]:
%%time
from openfermion.linalg import wedge

# 上記の操作をまとめたのが `wedge`関数である
left_index_ranks = (1, 1)
right_index_ranks = (2, 2)
thpdm = wedge(opdm, tpdm, left_index_ranks, right_index_ranks)

CPU times: user 38.2 ms, sys: 6.84 ms, total: 45 ms
Wall time: 43.4 ms


以下では、 `wedge`の実装部分を分解し、その動作を理解する。

In [13]:
from itertools import product
from math import factorial
from openfermion.linalg import generate_parity_permutations

# numpy einsum alphabet
EINSUM_CHARS = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

left_tensor = opdm
right_tensor = tpdm
left_index_ranks = (1, 1)
right_index_ranks = (2, 2)

# 巡回操作で現れる文字の整理
total_upper = left_index_ranks[0] + right_index_ranks[0]
total_lower = left_index_ranks[1] + right_index_ranks[1]
upper_characters = EINSUM_CHARS[:total_upper]
lower_characters = EINSUM_CHARS[total_upper:total_upper + total_lower]

# wedge積で生成される関数
new_tensor = np.zeros(left_tensor.shape + right_tensor.shape,
                         dtype=complex)
print("new_tensor.shape = ", new_tensor.shape)

new_tensor.shape =  (6, 6, 6, 6, 6, 6)


In [14]:
ordered_einsum_string = upper_characters + lower_characters
print(f"ordered_einsum_string = {ordered_einsum_string}")

# 巡回群全てについて和を計算
for upper_order_parities, lower_order_parities in product(
        generate_parity_permutations(upper_characters),
        generate_parity_permutations(lower_characters[::-1])):
    # we reverse the order in the lower_chars so because
    # <a^ b^ c d> = D_{dc}^{ab} in this code.
    n_upper_einsum_chars = upper_order_parities[0][:left_index_ranks[0]]
    m_upper_einsum_chars = upper_order_parities[0][left_index_ranks[0]:]
    n_lower_einsum_chars = lower_order_parities[0] \
        [:left_index_ranks[1]][::-1]
    m_lower_einsum_chars = lower_order_parities[0] \
        [left_index_ranks[1]:][::-1]

    n_string = "".join(n_upper_einsum_chars + n_lower_einsum_chars)
    m_string = "".join(m_upper_einsum_chars + m_lower_einsum_chars)
    
    #print("\nn_string = ", n_string)
    #print("m_string = ", m_string)
    n_parity = upper_order_parities[1]
    m_parity = lower_order_parities[1]
    
    print("einsum operation = {},{}->{}".format(n_string, m_string,ordered_einsum_string))
    print(f"sign = {n_parity * m_parity}")

    # we are doing lots of extra += operations but with the benefit of not
    # having to write a python loop over the entire new_tensor object.
    new_tensor += n_parity * m_parity * \
                  np.einsum('{},{}->{}'.format(n_string, m_string,
                                                  ordered_einsum_string),
                               left_tensor, right_tensor)

ordered_einsum_string = abcdef
einsum operation = af,bcde->abcdef
sign = 1
einsum operation = af,bced->abcdef
sign = -1
einsum operation = ad,bcef->abcdef
sign = 1
einsum operation = ae,bcdf->abcdef
sign = -1
einsum operation = ae,bcfd->abcdef
sign = 1
einsum operation = ad,bcfe->abcdef
sign = -1
einsum operation = af,cbde->abcdef
sign = -1
einsum operation = af,cbed->abcdef
sign = 1
einsum operation = ad,cbef->abcdef
sign = -1
einsum operation = ae,cbdf->abcdef
sign = 1
einsum operation = ae,cbfd->abcdef
sign = -1
einsum operation = ad,cbfe->abcdef
sign = 1
einsum operation = cf,abde->abcdef
sign = 1
einsum operation = cf,abed->abcdef
sign = -1
einsum operation = cd,abef->abcdef
sign = 1
einsum operation = ce,abdf->abcdef
sign = -1
einsum operation = ce,abfd->abcdef
sign = 1
einsum operation = cd,abfe->abcdef
sign = -1
einsum operation = bf,acde->abcdef
sign = -1
einsum operation = bf,aced->abcdef
sign = 1
einsum operation = bd,acef->abcdef
sign = -1
einsum operation = be,acdf->abcdef

In [15]:
# divided by number of permutations at the end
new_tensor /= factorial(total_upper) * factorial(total_lower)

In [16]:
# wedge関数のoutputと整合することを確認
np.allclose(thpdm, new_tensor)

True

## 実行時間テスト

(4, 4)階のテンソルの生成に時間がかかりすぎている。

In [17]:
%%time

# this is fine
unconnected_tpdm = wedge(opdm, opdm, (1, 1), (1,1))

CPU times: user 1.82 ms, sys: 1.57 ms, total: 3.39 ms
Wall time: 2 ms


In [18]:
%%time

# this is okay
unconnected_d3 = wedge(opdm, tpdm, (1, 1), (2, 2))

CPU times: user 23 ms, sys: 2 ms, total: 25 ms
Wall time: 22.4 ms


In [19]:
%%time

# generating (4, 4) tensor is too slow, even for 6 qubits
unconnected_d4 = wedge(tpdm, tpdm, (2, 2), (2, 2))

CPU times: user 16.8 s, sys: 88.9 ms, total: 16.9 s
Wall time: 16.9 s


In [20]:
%%time

# this is the case for other calculations as well
unconnected_d4_2 = wedge(opdm, unconnected_d3, (1, 1), (3, 3))

CPU times: user 16.8 s, sys: 45.1 ms, total: 16.9 s
Wall time: 16.9 s


### 10-qubit

In [21]:
%%time

# this is becoming irritating
unconnected_d3_10q = wedge(opdm_10q, tpdm_10q, (1, 1), (2, 2))

CPU times: user 579 ms, sys: 3.7 ms, total: 582 ms
Wall time: 582 ms


In [22]:
%%time

# slow as hell
unconnected_d4_10q = wedge(opdm_10q, unconnected_d3_10q, (1, 1), (3, 3))

KeyboardInterrupt: 

In [1]:
print("sugoi hayai program")

sugoi hayai program
